# MMD+GAN的一些想法

**理论上达到的效果:** 缓解GAN的model collapse，保证GAN生成数据的多样性。  
**思路:** 将MMD引入GAN的损失函数，作为GAN损失的正则化项。

## 1. MMD

**MMD:** 通过衡量两个数据集中样本的相似性，判断两个数据集中的样本是否来自于同一个分布。  

MMD公式：
$$M_{k}(\mathbb{P}, \mathbb{Q})=\left\|\mu_{\mathrm{P}}-\mu_{\mathrm{Q}}\right\|_{\mathcal{H}}^{2}=\mathbb{E}_{\mathrm{P}}\left[k\left(x, x^{\prime}\right)\right]-2 \mathbb{E}_{\mathrm{P}, Q}[k(x, y)]+\mathbb{E}_{\mathbb{Q}}\left[k\left(y, y^{\prime}\right)\right]$$  

其中k是核映射


## 2. GAN

GAN的损失：
$$\mathcal{L}_{\mathrm{GAN}}=\log (\operatorname{Dis}(\boldsymbol{x}))+\log (1-\operatorname{Dis}(\operatorname{Gen}(\boldsymbol{z})))$$

判别器目标：$D^* = \max_{Dis}L_{GAN}$  
生成器目标：$\min_{G} D^*$  
  
这样的GAN损失，会产生模式崩溃，导致生成样本多样性不足，并且生成样本和原有样本过于相似。  
### **所以，通过引入MMD，来度量生成样本和已有样本的相似性**。

## 3. MMD+GAN

很自然的想法是，将MMD作为GAN损失的正则化项，令GAN生成的样本和原有样本不能过于相似。但是，由于MMD是通过样本相似性来判断两个分布的相似性，所以若想将MMD作为GAN的损失项，需要计算生成样本和整个数据集的MMD指标，这就需要每次生成的样本足够多，并且计算量太大，显然行不通。所以退而求其次，我想通过minibatch的方法来度量生成样本和每个batch样本的相似性，这样就能够很简单的实现这种思想。暂且把这个想法叫做:BatchMMDGAN(BMMDGAN)

## BMMDGAN

原始MMD：
$$\mathcal{L}_{\mathrm{MMD}} \leftarrow 
        \left\|\frac{1}{N} \sum_{i=1}^{N} k\left(x_{i}\right)-\frac{1}{M} \sum_{j=1}^{M} k\left(y_{j}\right)\right\|^{2}$$

我们令$M=N=batch\_size$，所以就变为：$$\mathcal{L}_{\mathrm{MMD}} \leftarrow 
        \frac{1}{N} \sum_{i=1}^{N}\left\| k\left(x_{i}\right)-k\left(y_{j}\right)\right\|^{2}$$

## 算法流程

![jupyter](./BMMDGAN.png)